Nasa Space Apps Challenge 2025 Hackathan

A World Away: Hunting for Exoplanets with AI

Notebook 2 (2 of 2)

The dataset that is used to train the following models included in these notebooks comes from observations made by "NASA Kepler Space Telescope" and downloaded using "Kepler labelled time-series dataset on Kaggle". Each row represents a star’s flux measurements, with a label indicating whether a planet is present. The train dataset contains 5087 rows (Observations) and 3198 columns (features). Column 1 is the label vector and from columns 2 to 3198 are the flux values over time. There are 37 confirmed exoplanet-stars and 5050 non-exoplanet-stars in the train dataset. The challenge is to build a model that can identify exoplanets using their light curves. 

This notebook contains loading the trained model, preparing prediction function and building a simple web app using Gradio.

In [6]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr

# This defines the architecture of previously trained MLP model
class MLP(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )
    def forward(self, x):
        return self.net(x)

# This loads the saved trained model 
input_size = 3197
oversample_model = MLP(input_size)
oversample_model.load_state_dict(torch.load("oversample_model.pth", map_location=torch.device("cpu")))
oversample_model.eval()

labels = ["No Planet (0)", "Planet (1)"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# This function plots the light curve and predicts the class
def predict_lightcurve(row_values):
    # Convert comma-separated string to float array
    X = np.array([float(x) for x in row_values.split(',')], dtype=np.float32)
    
    # This standardizes input (zero mean, unit variance)
    X_mean = X.mean()
    X_std = X.std()
    if X_std > 0:
        X = (X - X_mean) / X_std
    else:
        X = X - X_mean 

    X_tensor = torch.tensor(X).unsqueeze(0).to(device)

    with torch.no_grad():
        out = oversample_model(X_tensor)
        probs = torch.softmax(out, dim=1).cpu().numpy().flatten()

    pred_class = labels[np.argmax(probs)]

    # This plots the light curve
    fig, ax = plt.subplots(figsize=(10, 3))
    ax.plot(X, color="blue")
    ax.set_title(
        f"Prediction: {pred_class} | No Planet={probs[0]:.3f}, Planet={probs[1]:.3f}"
    )
    ax.set_xlabel("Time steps")
    ax.set_ylabel("Flux")

    return (
        f"Predicted: {pred_class}\n"
        f"Probabilities -> No Planet: {probs[0]:.3f}, Planet: {probs[1]:.3f}",
        fig
    )

# This is the GUI using Gradio
iface = gr.Interface(
    fn=predict_lightcurve,
    inputs=gr.Textbox(lines=5, placeholder="Enter comma-separated flux values"),
    outputs=[
        "text",
        gr.Plot()
    ],
    title="Exoplanet Detector",
    description="Paste a light curve (flux values) and predict if it's a planet or not. Also see the flux curve plotted."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Created dataset file at: .gradio\flagged\dataset1.csv
